In [14]:
import pathlib

import pandas as pd
import spacy
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

BASE_DIR = pathlib.Path.cwd().parent.parent.resolve()
BASE_DIR

WindowsPath('C:/Users/Alejandro Feria/Desktop/Unal/8 semestre/pnl/SilkNews')

### Cargar dataset

In [15]:
clean_df = pd.read_csv(f"{BASE_DIR}/src/data/clean_data/noticias_combinadas.csv")
clean_df['text'] = clean_df['text'].astype(str)
print(clean_df.shape)

df = clean_df
print(df.shape)

df.head()

(5568, 2)
(5568, 2)


,text,label
0,en una rueda de prensa las autoridades dijeron...,True
1,una operacion en la que las autoridades irland...,True
2,el nuevo informe se basa en mas de entrevistas...,True
3,las pruebas hechas a varias de las principales...,True
4,michelle y john wylie estuvieron buscando dura...,True


In [16]:
# uv run --with pip spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")
nlp

## tokenizar y aplicar stemming

In [17]:
stemmer = SnowballStemmer('spanish')
def tokenize(text: str) -> list[str]:
  return " ".join([stemmer.stem(token.text) for token in nlp(text) if not token.is_punct and not token.is_space])

df["texto_limpio"] = df["text"].apply(tokenize)

In [18]:
df.head()

,text,label,texto_limpio
0,en una rueda de prensa las autoridades dijeron...,True,en una rued de prens las autor dijeron que los...
1,una operacion en la que las autoridades irland...,True,una oper en la que las autor irlandes decomis ...
2,el nuevo informe se basa en mas de entrevistas...,True,el nuev inform se bas en mas de entrev con per...
3,las pruebas hechas a varias de las principales...,True,las prueb hech a vari de las principal marc ar...
4,michelle y john wylie estuvieron buscando dura...,True,michell y john wyli estuv busc durant anos a u...


In [19]:
# Bolsa de Palabras, unigramas y bigramas
def bag_of_words_sin_stopwords(texto_limpio: str) -> list[str]:
  vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=list(nlp.Defaults.stop_words))
  vectorizer.fit_transform([texto_limpio])
  return vectorizer.vocabulary_

def bag_of_words_frecuencia(texto_limpio: str) -> list[str]:
    vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=list(nlp.Defaults.stop_words))
    vectorizer.fit_transform([texto_limpio])

    vocabulario_eliminacion_frecuencia = {}
    # Filtrar el vocabulario para eliminar palabras con frecuencia < 3
    for palabra, indice in vectorizer.vocabulary_.items():
        if vectorizer.vocabulary_[palabra] >= 3:
            vocabulario_eliminacion_frecuencia[palabra] = indice

    return vocabulario_eliminacion_frecuencia

df["sin_stopwords"] = df["texto_limpio"].apply(bag_of_words_sin_stopwords)
df["frecuencia<3"] = df["texto_limpio"].apply(bag_of_words_frecuencia)

In [20]:
df

,text,label,texto_limpio,sin_stopwords,frecuencia<3
0,en una rueda de prensa las autoridades dijeron...,True,en una rued de prens las autor dijeron que los...,"{'rued': 19, 'prens': 13, 'autor': 0, 'familia...","{'rued': 19, 'prens': 13, 'familiar': 6, 'robi..."
1,una operacion en la que las autoridades irland...,True,una oper en la que las autor irlandes decomis ...,"{'oper': 17, 'autor': 0, 'irlandes': 13, 'deco...","{'oper': 17, 'irlandes': 13, 'decomis': 4, 'to..."
2,el nuevo informe se basa en mas de entrevistas...,True,el nuev inform se bas en mas de entrev con per...,"{'nuev': 13, 'inform': 9, 'bas': 1, 'entrev': ...","{'nuev': 13, 'inform': 9, 'entrev': 5, 'person..."
3,las pruebas hechas a varias de las principales...,True,las prueb hech a vari de las principal marc ar...,"{'prueb': 9, 'hech': 2, 'vari': 13, 'principal...","{'prueb': 9, 'vari': 13, 'principal': 7, 'marc..."
4,michelle y john wylie estuvieron buscando dura...,True,michell y john wyli estuv busc durant anos a u...,"{'michell': 15, 'john': 13, 'wyli': 21, 'estuv...","{'michell': 15, 'john': 13, 'wyli': 21, 'estuv..."
...,...,...,...,...,...
5563,manuela carmena pide a los madrilenos que guar...,False,manuel carmen pid a los madrilen que guard las...,"{'manuel': 15, 'carmen': 4, 'pid': 19, 'madril...","{'manuel': 15, 'carmen': 4, 'pid': 19, 'madril..."
5564,a solo unas horas de dejar el poder el expresi...,False,a sol unas hor de dej el pod el expresident en...,"{'sol': 306, 'hor': 170, 'dej': 72, 'pod': 269...","{'sol': 306, 'hor': 170, 'dej': 72, 'pod': 269..."
5565,hartos de ser senalados como unos eternos segu...,False,hart de ser senal com unos etern segundon sin ...,"{'hart': 172, 'senal': 350, 'com': 59, 'etern'...","{'hart': 172, 'senal': 350, 'com': 59, 'etern'..."
5566,multan a un ultrasur por lanzar a un jugador d...,False,mult a un ultrasur por lanz a un jugador del b...,"{'mult': 117, 'ultrasur': 160, 'lanz': 93, 'ju...","{'mult': 117, 'ultrasur': 160, 'lanz': 93, 'ju..."


In [21]:
texto = "el zorro marrón salta sobre el perro perezoso. El zorro es muy rápido y astuto."
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=list(nlp.Defaults.stop_words))
vectorizer.fit_transform([texto])

vectorizer.vocabulary_

{'zorro': 11,
 'marrón': 1,
 'salta': 9,
 'perro': 5,
 'perezoso': 3,
 'rápido': 7,
 'astuto': 0,
 'zorro marrón': 12,
 'marrón salta': 2,
 'salta perro': 10,
 'perro perezoso': 6,
 'perezoso zorro': 4,
 'zorro rápido': 13,
 'rápido astuto': 8}

In [22]:
df

,text,label,texto_limpio,sin_stopwords,frecuencia<3
0,en una rueda de prensa las autoridades dijeron...,True,en una rued de prens las autor dijeron que los...,"{'rued': 19, 'prens': 13, 'autor': 0, 'familia...","{'rued': 19, 'prens': 13, 'familiar': 6, 'robi..."
1,una operacion en la que las autoridades irland...,True,una oper en la que las autor irlandes decomis ...,"{'oper': 17, 'autor': 0, 'irlandes': 13, 'deco...","{'oper': 17, 'irlandes': 13, 'decomis': 4, 'to..."
2,el nuevo informe se basa en mas de entrevistas...,True,el nuev inform se bas en mas de entrev con per...,"{'nuev': 13, 'inform': 9, 'bas': 1, 'entrev': ...","{'nuev': 13, 'inform': 9, 'entrev': 5, 'person..."
3,las pruebas hechas a varias de las principales...,True,las prueb hech a vari de las principal marc ar...,"{'prueb': 9, 'hech': 2, 'vari': 13, 'principal...","{'prueb': 9, 'vari': 13, 'principal': 7, 'marc..."
4,michelle y john wylie estuvieron buscando dura...,True,michell y john wyli estuv busc durant anos a u...,"{'michell': 15, 'john': 13, 'wyli': 21, 'estuv...","{'michell': 15, 'john': 13, 'wyli': 21, 'estuv..."
...,...,...,...,...,...
5563,manuela carmena pide a los madrilenos que guar...,False,manuel carmen pid a los madrilen que guard las...,"{'manuel': 15, 'carmen': 4, 'pid': 19, 'madril...","{'manuel': 15, 'carmen': 4, 'pid': 19, 'madril..."
5564,a solo unas horas de dejar el poder el expresi...,False,a sol unas hor de dej el pod el expresident en...,"{'sol': 306, 'hor': 170, 'dej': 72, 'pod': 269...","{'sol': 306, 'hor': 170, 'dej': 72, 'pod': 269..."
5565,hartos de ser senalados como unos eternos segu...,False,hart de ser senal com unos etern segundon sin ...,"{'hart': 172, 'senal': 350, 'com': 59, 'etern'...","{'hart': 172, 'senal': 350, 'com': 59, 'etern'..."
5566,multan a un ultrasur por lanzar a un jugador d...,False,mult a un ultrasur por lanz a un jugador del b...,"{'mult': 117, 'ultrasur': 160, 'lanz': 93, 'ju...","{'mult': 117, 'ultrasur': 160, 'lanz': 93, 'ju..."


In [23]:
df.to_csv(f"{BASE_DIR}/src/data/extraccion_caracteristicas/extraccion_caracteristicas.csv", index=False)